# Zone Scan

A crystallographic **zone** is characterized by a set of planes which are all
parallel to one line, called the *zone axis*. (*)  If the axis of a zone has indices
{math}`[uvw]`, then any plane belongs to that zone whose indices {math}`(hkl)`
satisfy the relation:

{math}`hu+hv+lw=0`

In diffractometer use, a zone is used to align, index, and interpret diffraction patterns. Key uses and practical steps:

## Purpose
 - *Crystal Alignment*: Orient the sample so a chosen zone axis is parallel
   to the incident beam or goniometer rotation axis. That places high symmetry
   directions into the diffractometer frame and often maximizes systematic
   reflections.
- *Indexing*: When the crystal is on a zone axis, all observed
  diffraction spots correspond to reflections whose reciprocal-lattice vectors
  lie in a plane perpendicular to that axis. This simplifies assignment of
  Miller indices.
- *Symmetry determination*: A zone-axis pattern reveals symmetry of the lattice
  projection, helping determine lattice type, possible space groups, and
  systematic absences.
- *Setting up data collection*: Choosing zone axes for multiple orientations
  (e.g., principal axes) helps plan rotation ranges and ensures coverage of
  independent reflections.


(*) *Elements of X-ray Diffraction*, B.D. Cullity, 1978, Second Edition.

In [1]:
import numpy as np
from hklpy2.blocks.zone import OrthonormalZone, zone_series, scan_zone

np.set_printoptions(precision=4, suppress=True, floatmode="maxprec")

In [2]:
v1 = np.array((1, 1, 0))
v2 = np.array((-3, -2, -1))
v3 = np.array((-1, 1, 1))

zone = OrthonormalZone()
print(f"{zone=}")
zone.axis = v3
print(f"{zone=}")
print(f"{OrthonormalZone(axis=v1)=}")
print(f"{OrthonormalZone(axis=[0, 1, 1])=}")
print(f"{OrthonormalZone(axis=(1, 2, 3))=}")
print(f"{OrthonormalZone(axis=dict(u=11, v=22, w=33))=}")
print(f"{OrthonormalZone(v1=v1, v2=v2)=}")

print(f"{zone.in_zone(v1)=}")
print(f"{zone.in_zone(v2)=}")
print(f"{zone.in_zone(v3)=}")

print(f"{v1=}")
print(f"{v2=}")

zone=OrthonormalZone(axis='undefined')
zone=OrthonormalZone(axis=array([-0.5774,  0.5774,  0.5774]))
OrthonormalZone(axis=v1)=OrthonormalZone(axis=array([0.7071, 0.7071, 0.    ]))
OrthonormalZone(axis=[0, 1, 1])=OrthonormalZone(axis=array([0.    , 0.7071, 0.7071]))
OrthonormalZone(axis=(1, 2, 3))=OrthonormalZone(axis=array([0.2673, 0.5345, 0.8018]))
OrthonormalZone(axis=dict(u=11, v=22, w=33))=OrthonormalZone(axis=array([0.2673, 0.5345, 0.8018]))


TypeError: OrthonormalZone.__init__() got an unexpected keyword argument 'v1'. Did you mean 'b1'?

In [ ]:
import hklpy2

sim = hklpy2.creator()
print(f"{sim.forward([-0.5, -1., 1.])=}")
print(f"{sim.forward([0, -1, 0.5])=}")

In [ ]:
# zone_series(sim, v1, v3, 17)
# zone_series(sim, (1, 0, 0), [0, -1, 0.5], 17)
sim.core.constraints["tth"].limits = -0.01, 180.01
zone_series(sim, (1, 0, 0), [0, 1, 0], 5)

In [ ]:
sim.add_sample("test", 4, 5, 6, 75, 85, 95, replace=True)
r1 = sim.add_reflection((4, 0, 0), (30.345, 10, 10, 60.69))
r2 = sim.add_reflection((0, 4, 0), (-24.63, -9.265, -85.08, -49.27))
np.asarray(sim.core.calc_UB(r1, r2), dtype=float)

In [ ]:
zone_series(sim, (4, 0, 0), [0, 4, 0], 5)

In [ ]:
from bluesky import RunEngine
from bluesky.callbacks.best_effort import BestEffortCallback
from ophyd.sim import noisy_det

RE = RunEngine()
bec = BestEffortCallback()
bec.disable_plots()
RE.subscribe(bec)

In [ ]:
uid, = RE(scan_zone([noisy_det], sim, (1,0,0), (0,1,0), 5))